# Phase 2: Instruction Tuning 데이터셋 구축

Phase 1에서 분석한 인기글 데이터를 활용해 Instruction Tuning용 JSONL 데이터셋을 생성합니다.


## ✅ 목표
- 인기글 정제 데이터(`인기글_clean.parquet`) 불러오기
- 게시판명 정규화 (`board_clean`)
- 키워드 추출(Mecab 기반, 불용어 제거)
- Instruction/Input/Output 포맷으로 JSONL 생성 (`outputs/training_dataset.jsonl`)
- 샘플 레코드 프린트로 검증


## ⚙️ 사전 조건
- `phase0_setup.ipynb` 실행으로 `outputs/인기글_clean.parquet` 존재
- (선택) `phase1_topic_modeling.ipynb` 실행으로 `인기글_topics_for_prompt.json` 등 추가 메타 활용 가능
- Colab 런타임 초기화 시 아래 패키지 설치 셀을 다시 실행


---
## 0. 패키지 설치 (필요 시)
- Mecab/konlpy가 설치되어 있지 않다면 먼저 설치하세요.
- 만약 설치에 실패하더라도, 이 노트북은 자동으로 Mecab 토크나이저를 건너뛰고 **문자열 기반 키워드 추출 방식**으로 fallback 하도록 구현되어 있습니다.


In [ ]:
%pip install -q pandas numpy konlpy mecab-python3

print("✅ 기본 패키지 설치 확인")


In [ ]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


---
## 1. 경로 설정 및 데이터 로드


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

HOT_PARQUET = OUTPUT_DIR / "인기글_clean.parquet"
if not HOT_PARQUET.exists():
    raise FileNotFoundError(f"{HOT_PARQUET}를 찾을 수 없습니다. Phase 0을 먼저 실행하세요.")

hot_df = pd.read_parquet(HOT_PARQUET)
print(f"✅ 인기글 데이터 로드 완료: {len(hot_df):,}행")


---
## 2. 게시판명 정규화


In [ ]:
BOARD_CANONICAL_MAP = {
    "익게2": "익게2",
    "|익게2": "익게2",
    "익명2": "익게2",
    "익명게2": "익게2",
    "익게1": "익게1",
    "|익게1": "익게1",
    "연애": "연애상담소",
    "연애상담소": "연애상담소",
    "자유": "자유게시판",
    "자유게시판": "자유게시판",
}

BOARD_FALLBACK_RULES = {
    "ik2": "익게2",
    "ik-2": "익게2",
    "ik1": "익게1",
    "love": "연애상담소",
    "free": "자유게시판",
}


def normalize_board_name(raw_value: str, url_value: str) -> str:
    raw_value = (raw_value or "").strip().replace("|", "").replace("#", "")
    if raw_value in BOARD_CANONICAL_MAP:
        return BOARD_CANONICAL_MAP[raw_value]

    lowered_raw = raw_value.lower()
    for key, target in BOARD_FALLBACK_RULES.items():
        if key in lowered_raw:
            return target

    lowered_url = (url_value or "").lower()
    for key, target in BOARD_FALLBACK_RULES.items():
        if key in lowered_url:
            return target

    return raw_value or "미상"

hot_df["board_clean"] = hot_df.apply(
    lambda row: normalize_board_name(row.get("board", ""), row.get("url", "")),
    axis=1,
)
print("📊 게시판 분포:")
print(hot_df["board_clean"].value_counts().head(10))


---
## 3. 키워드 추출 함수 (Mecab + 불용어)


In [ ]:
try:
    from konlpy.tag import Mecab
    mecab = Mecab()
    MECAB_AVAILABLE = True
    print("✅ Mecab 사용 가능")
except Exception as e:
    MECAB_AVAILABLE = False
    mecab = None
    print(f"⚠️ Mecab 초기화 실패: {e}")

import re
from collections import Counter

DEFAULT_STOPWORDS = [
    "이","가","을","를","에","의","와","과","도","로","으로",
    "은","는","에서","에게","께","한테",
    "더","그","저","이것","그것","저것","그런","이런","저런","그렇","이렇","저렇",
    "때","때문","것","수","등",
    "있다","없다","하다","되다","이다","아니다","같다",
    "게시판","게시글","글","댓글","작성","작성자","조회","추천","비추천"
]
CUSTOM_STOPWORDS = set(DEFAULT_STOPWORDS)


def extract_keywords_from_text(text: str, top_k: int = 8,
                               pos_filter=('NNG', 'NNP', 'VV', 'VA', 'MM'),
                               min_len: int = 2):
    if pd.isna(text) or not text:
        return []

    tokens = []
    if MECAB_AVAILABLE and mecab is not None:
        try:
            tokens = [
                word for word, pos in mecab.pos(text)
                if (pos_filter is None or pos in pos_filter) and len(word) >= min_len
            ]
        except Exception as exc:
            print(f"⚠️ Mecab 토큰화 실패: {exc}")
            tokens = re.findall(r"[가-힣a-zA-Z0-9]+", text)
    else:
        tokens = re.findall(r"[가-힣a-zA-Z0-9]+", text)

    filtered = [tok for tok in tokens if tok not in CUSTOM_STOPWORDS]
    counts = Counter(filtered)
    return [token for token, _ in counts.most_common(top_k)]


def attach_keywords(df: pd.DataFrame, top_k: int = 8) -> pd.DataFrame:
    df = df.copy()
    combined = df["title"].fillna("") + " " + df["contents"].fillna("")
    df["keyword_list"] = combined.apply(lambda text: extract_keywords_from_text(text, top_k=top_k))
    df["keywords"] = df["keyword_list"].apply(lambda ks: ", ".join(ks))
    return df


In [ ]:
hot_with_keywords = attach_keywords(hot_df, top_k=8)
print("추출된 키워드 미리보기:")
display(hot_with_keywords[["id", "board_clean", "keywords"]].head(5))


In [ ]:
hot_with_keywords_path = OUTPUT_DIR / "인기글_with_keywords.parquet"
hot_with_keywords.to_parquet(hot_with_keywords_path, index=False)
print(f"💾 키워드 부착 데이터 저장: {hot_with_keywords_path}")


---
## 4. Instruction Tuning JSONL 생성


In [ ]:
import json
from datetime import datetime

def create_instruction_dataset(df: pd.DataFrame, output_path: Path, sample_size: int = None):
    records = []
    source_df = df if sample_size is None else df.sample(n=sample_size, random_state=42)
    for _, row in source_df.iterrows():
        keywords = row.get("keyword_list", [])
        if not keywords:
            continue

        board_name = row.get("board_clean", "미상")
        title = str(row.get("title", "")).strip()
        contents = str(row.get("contents", "")).strip()
        full_text = f"{title}\n\n{contents}".strip()
        if not full_text:
            continue

        instruction = f"주어진 키워드를 사용하여 {board_name}의 인기글 스타일로 글을 작성해줘."
        input_text = f"키워드: {', '.join(keywords)}"

        records.append({
            "instruction": instruction,
            "input": input_text,
            "output": full_text,
        })

    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        for record in records:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

    print(f"✅ JSONL 생성 완료: {len(records)}개 레코드 → {output_path}")
    return records


In [ ]:
jsonl_path = OUTPUT_DIR / "training_dataset.jsonl"
training_records = create_instruction_dataset(hot_with_keywords, jsonl_path)

print("\n샘플 데이터 (최대 3건):")
for i, rec in enumerate(training_records[:3], 1):
    print(f"\n[{i}]")
    print(f"Instruction: {rec['instruction']}")
    print(f"Input: {rec['input']}")
    print(f"Output (앞 120자): {rec['output'][:120]}...")


---
## 5. 완료 및 다음 단계
- 생성된 `training_dataset.jsonl`은 Phase 3 파인튜닝 노트북에서 그대로 사용할 수 있습니다.
- 필요 시 `hot_with_keywords` Parquet을 참고하여 추가 필터링/샘플링을 수행하세요.
- 다음 단계: `phase3_finetune.ipynb`에서 QLoRA 학습 진행
